In [1]:
import numpy as np
import torch 
import matplotlib.pyplot as plt
import os
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader,random_split
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch import optim
from pytorch_lightning.core import LightningModule
from pytorch_lightning.metrics.functional import  accuracy 
from pytorch_lightning import loggers as pl_loggers
from torchvision import datasets, transforms
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import string

C:\Users\mohamedr002\Anaconda3\envs\phd\lib\site-packages\pytorch_lightning\utilities\distributed.py:37: UserWarning: Unsupported `ReduceOp` for distributed computing.
  warnings.warn(*args, **kwargs)


In [2]:
class HAR_torch_Dataset(Dataset):
    def __init__(self, data,labels, transform= None):
        """Reads source and target sequences from processing file ."""        
        self.input_tensor = (torch.from_numpy(data)).float()
        self.label = torch.LongTensor(labels)
        self.transform =transform
        self.num_total_seqs = len(self.input_tensor)

    def __getitem__(self, index):
        """Returns one data pair (source and target)."""
        input_seq = self.input_tensor[index]
        input_labels = self.label[index]
        if self.transform:
            input_seq = self.transform(input_seq)
        return input_seq, input_labels

    def __len__(self):
        return self.num_total_seqs

def HAR_data_generator(data_dir,save=False):
    # dataloading 
    domains_data = []
    domains_labels = []
    subject_data= np.loadtxt(f'{data_dir}/train/subject_train.txt')
    # Samples
    train_acc_x= np.loadtxt(f'{data_dir}/train/Inertial Signals/body_acc_x_train.txt')
    train_acc_y= np.loadtxt(f'{data_dir}/train/Inertial Signals/body_acc_y_train.txt')
    train_acc_z= np.loadtxt(f'{data_dir}/train/Inertial Signals/body_acc_z_train.txt')
    train_gyro_x= np.loadtxt(f'{data_dir}/train/Inertial Signals/body_gyro_x_train.txt')
    train_gyro_y= np.loadtxt(f'{data_dir}/train/Inertial Signals/body_gyro_y_train.txt')
    train_gyro_z= np.loadtxt(f'{data_dir}/train/Inertial Signals/body_gyro_z_train.txt')
    train_tot_acc_x= np.loadtxt(f'{data_dir}/train/Inertial Signals/total_acc_x_train.txt')
    train_tot_acc_y= np.loadtxt(f'{data_dir}/train/Inertial Signals/total_acc_y_train.txt')
    train_tot_acc_z= np.loadtxt(f'{data_dir}/train/Inertial Signals/total_acc_z_train.txt')

    test_acc_x= np.loadtxt(f'{data_dir}/test/Inertial Signals/body_acc_x_test.txt')
    test_acc_y= np.loadtxt(f'{data_dir}/test/Inertial Signals/body_acc_y_test.txt')
    test_acc_z= np.loadtxt(f'{data_dir}/test/Inertial Signals/body_acc_z_test.txt')
    test_gyro_x= np.loadtxt(f'{data_dir}/test/Inertial Signals/body_gyro_x_test.txt')
    test_gyro_y= np.loadtxt(f'{data_dir}/test/Inertial Signals/body_gyro_y_test.txt')
    test_gyro_z= np.loadtxt(f'{data_dir}/test/Inertial Signals/body_gyro_z_test.txt')
    test_tot_acc_x= np.loadtxt(f'{data_dir}/test/Inertial Signals/total_acc_x_test.txt')
    test_tot_acc_y= np.loadtxt(f'{data_dir}/test/Inertial Signals/total_acc_y_test.txt')
    test_tot_acc_z= np.loadtxt(f'{data_dir}/test/Inertial Signals/total_acc_z_test.txt')

    # Stacking channels together data 
    train_data= np.stack((train_acc_x,train_acc_y,train_acc_z,
                              train_gyro_x,train_gyro_y,train_gyro_z,
                              train_tot_acc_x, train_tot_acc_y,train_tot_acc_z),axis=1)
    test_data= np.stack((test_acc_x,test_acc_y,test_acc_z,
                              test_gyro_x,test_gyro_y,test_gyro_z,
                              test_tot_acc_x, test_tot_acc_y,test_tot_acc_z),axis=1)
    # labels 
    train_labels=  np.loadtxt(f'{data_dir}/train/y_train.txt')
    train_labels -= np.min(train_labels)
    test_labels=  np.loadtxt(f'{data_dir}/test/y_test.txt')
    test_labels -= np.min(test_labels)

    # different subjects 
    subject_train= np.loadtxt(f'{data_dir}/train/subject_train.txt')
    subject_test= np.loadtxt(f'{data_dir}/test/subject_test.txt')
    # select subset i for train and subset j for testing

    all_subjects_data = np.concatenate((train_data, test_data))
    all_subjects_labels = np.concatenate((train_labels, test_labels))
    subject_indices =  np.concatenate((subject_train, subject_test))
    # arrange the subjects to different domains
    print(subject_indices)
    subject_list = [1,3,5,]
    domain_names = list(string.ascii_lowercase)
    for i in range(1, 30):
        domains_data.append(all_subjects_data[np.where(subject_indices==i)])
        domains_labels.append(all_subjects_labels[np.where(subject_indices==i)])

    # split the domains to train_val_test
    for domain_data, domain_labels, name in zip(domains_data, domains_labels, domain_names):
        # train, validation, test split of the data 
        print(domain_data.shape)
        X_train, X_test, y_train, y_test = train_test_split(domain_data, domain_labels, test_size=0.2, random_state=1)

        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

        train = {'samples':X_train, 'labels':y_train}
        val   = {'samples':X_val, 'labels':y_val}
        test   = {'samples':X_test, 'labels':y_test}
        torch.save(train, f'train_{name}.pt')
        torch.save(val,f'val_{name}.pt')
        torch.save(test,f'test_{name}.pt')

In [ ]:
domain_names = np.arange(1,2)

In [5]:
data_dir = '../../UCI HAR Dataset'
HAR_data_generator(data_dir)

[ 1.  1.  1. ... 24. 24. 24.]
(347, 9, 128)
(302, 9, 128)
(341, 9, 128)
(317, 9, 128)
(302, 9, 128)
(325, 9, 128)
(308, 9, 128)
(281, 9, 128)
(288, 9, 128)
(294, 9, 128)
(316, 9, 128)
(320, 9, 128)
(327, 9, 128)
(323, 9, 128)
(328, 9, 128)
(366, 9, 128)
(368, 9, 128)
(364, 9, 128)
(360, 9, 128)
(354, 9, 128)
(408, 9, 128)
(321, 9, 128)
(372, 9, 128)
(381, 9, 128)


In [8]:
all_subjects_data.shape

NameError: name 'all_subjects_data' is not defined

In [ ]:
# arrange the subjects to different domains
domain_names = list(string.ascii_lowercase)
for i in range(0, 25):
    domains_data.append(all_subjects_data[i])
    domains_labels.append(all_subjects_labels[i])

# split the domains to train_val_test
for domain_data, domain_labels, name in zip(domains_data, domains_labels, domain_names):
    # train, validation, test split of the data 
    X_train, X_test, y_train, y_test = train_test_split(domain_data, domain_labels, test_size=0.2, random_state=1)

    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)
    
    train = {'samples':X_train, 'labels':y_train}
    val   = {'samples':X_val, 'labels':y_val}
    test   = {'samples':X_test, 'labels':y_test}
    torch.save(train, f'train_{name}.pt')
    torch.save(val,f'val_{name}.pt')
    torch.save(test,f'test_{name}.pt')

Data Preparation 

In [ ]:
def HAR_data_loader(dataset, batch_size, shuffle, drop_last) :
    # datasets 
    train_dataset = HAR_torch_Dataset(dataset['train']['samples'], dataset['train']['labels'])
    val_dataset = HAR_torch_Dataset(dataset['val']['samples'], dataset['val']['labels'])
    test_dataset = HAR_torch_Dataset(dataset['test']['samples'], dataset['test']['labels'])
    # dataloaders
    train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    valid_dl = DataLoader(val_dataset, batch_size=10, shuffle=False, drop_last=False)
    test_dl = DataLoader(test_dataset, batch_size=10, shuffle=False, drop_last=False)

    return train_dl, valid_dl, test_dl

In [4]:
# dataloading 
data_dir = r'D:\My TS Datasets\HAR_Datasets\HAR\UCI HAR Dataset'
domains_data = []
domains_labels = []
subject_data= np.loadtxt(f'{data_dir}/train/subject_train.txt')
# Samples
train_acc_x= np.loadtxt(f'{data_dir}/train/Inertial Signals/body_acc_x_train.txt')
train_acc_y= np.loadtxt(f'{data_dir}/train/Inertial Signals/body_acc_y_train.txt')
train_acc_z= np.loadtxt(f'{data_dir}/train/Inertial Signals/body_acc_z_train.txt')
train_gyro_x= np.loadtxt(f'{data_dir}/train/Inertial Signals/body_gyro_x_train.txt')
train_gyro_y= np.loadtxt(f'{data_dir}/train/Inertial Signals/body_gyro_y_train.txt')
train_gyro_z= np.loadtxt(f'{data_dir}/train/Inertial Signals/body_gyro_z_train.txt')
train_tot_acc_x= np.loadtxt(f'{data_dir}/train/Inertial Signals/total_acc_x_train.txt')
train_tot_acc_y= np.loadtxt(f'{data_dir}/train/Inertial Signals/total_acc_y_train.txt')
train_tot_acc_z= np.loadtxt(f'{data_dir}/train/Inertial Signals/total_acc_z_train.txt')

test_acc_x= np.loadtxt(f'{data_dir}/test/Inertial Signals/body_acc_x_test.txt')
test_acc_y= np.loadtxt(f'{data_dir}/test/Inertial Signals/body_acc_y_test.txt')
test_acc_z= np.loadtxt(f'{data_dir}/test/Inertial Signals/body_acc_z_test.txt')
test_gyro_x= np.loadtxt(f'{data_dir}/test/Inertial Signals/body_gyro_x_test.txt')
test_gyro_y= np.loadtxt(f'{data_dir}/test/Inertial Signals/body_gyro_y_test.txt')
test_gyro_z= np.loadtxt(f'{data_dir}/test/Inertial Signals/body_gyro_z_test.txt')
test_tot_acc_x= np.loadtxt(f'{data_dir}/test/Inertial Signals/total_acc_x_test.txt')
test_tot_acc_y= np.loadtxt(f'{data_dir}/test/Inertial Signals/total_acc_y_test.txt')
test_tot_acc_z= np.loadtxt(f'{data_dir}/test/Inertial Signals/total_acc_z_test.txt')

# Stacking channels together data 
train_data= np.stack((train_acc_x,train_acc_y,train_acc_z,
                          train_gyro_x,train_gyro_y,train_gyro_z,
                          train_tot_acc_x, train_tot_acc_y,train_tot_acc_z),axis=1)
test_data= np.stack((test_acc_x,test_acc_y,test_acc_z,
                          test_gyro_x,test_gyro_y,test_gyro_z,
                          test_tot_acc_x, test_tot_acc_y,test_tot_acc_z),axis=1)
# labels 
train_labels=  np.loadtxt(f'{data_dir}/train/y_train.txt')
train_labels -= np.min(train_labels)
test_labels=  np.loadtxt(f'{data_dir}/test/y_test.txt')
test_labels -= np.min(test_labels)

# different subjects 
subject_train= np.loadtxt(f'{data_dir}/train/subject_train.txt')
subject_test= np.loadtxt(f'{data_dir}/test/subject_test.txt')
# select subset i for train and subset j for testing

all_subjects_data = np.concatenate((train_data, test_data))
all_subjects_labels = np.concatenate((train_labels, test_labels))
subject_indices =  np.concatenate((subject_train, subject_test))
# arrange the subjects to different domains

for i in range(1, 31):
    subject = all_subjects_data[np.where(subject_indices==i)]
    label = all_subjects_labels[np.where(subject_indices==i)]
    
    X_train, X_test, y_train, y_test = train_test_split(subject, label, test_size=0.3, stratify=label, random_state=1)
#     X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25,  stratify=True, random_state=1)

    train = {'samples':X_train, 'labels':y_train}
#     val   = {'samples':X_val, 'labels':y_val}
    test   = {'samples':X_test, 'labels':y_test}
    torch.save(train, f'train_{i}.pt')
#     torch.save(val,f'val_{i}.pt')
    torch.save(test,f'test_{i}.pt')

In [8]:
y= y_train.tolist()
print([y.count(i) for i in range(6)])
y= y_test.tolist()
print([y.count(i) for i in range(6)])

[46, 46, 43, 43, 41, 49]
[19, 19, 19, 19, 18, 21]
